In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
import os
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import EfficientNetB1
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, ModelCheckpoint
from sklearn.metrics import classification_report,confusion_matrix
import io
from PIL import Image

In [ ]:
labels = ["cat", "dog"]

In [ ]:
X_train = []
y_train = []
img_size = 100

folderPathDog = os.path.join(r"/content/PetImages/Dog/")
folderPathCat = os.path.join(r"/content/PetImages/Cat/")

for j in tqdm(os.listdir(folderPathDog)):
    img = cv2.imread(os.path.join(folderPathDog, j))
    img = cv2.resize(img, (img_size, img_size))
    X_train.append(img)
    y_train.append(1)

for j in tqdm(os.listdir(folderPathCat)):
    if j[-3:-1] == "jp":
      img = cv2.imread(os.path.join(folderPathCat, j))
      img = cv2.resize(img, (img_size, img_size))
      X_train.append(img)
      y_train.append(0)

X_train = np.array(X_train)
y_train = np.array(y_train)

100%|██████████| 501/501 [00:01<00:00, 376.51it/s]


In [ ]:
X_train.shape

(1001, 100, 100, 3)

In [ ]:
X_train, y_train = shuffle(X_train, y_train, random_state=101)

In [ ]:
y_train[0]

1

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.1, random_state=101)

In [ ]:
y_train = tf.keras.utils.to_categorical(y_train)

In [ ]:
y_train

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [0., 1.],
       [1., 0.]], dtype=float32)

In [ ]:
effnet = EfficientNetB1(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

27018416/27018416 [==============================] - 0s 0us/step


In [ ]:
model = effnet.output
model = tf.keras.layers.GlobalAveragePooling2D()(model)
model = tf.keras.layers.Dropout(rate=0.5)(model)
model = tf.keras.layers.Dense(2,activation='softmax')(model)
model = tf.keras.models.Model(inputs=effnet.input, outputs = model)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer= 'Adam', metrics= ['accuracy'])

In [ ]:
tensorboard = TensorBoard(log_dir = 'logs')
checkpoint = ModelCheckpoint('effnetB1_V2.h5', monitor='val_accuracy', save_best_only=True, mode='auto', verbose=1)
reduce_lr = ReduceLROnPlateau(monitor= 'val_accuracy', factor=0.3, patience=2, min_delta=0.001, mode='auto', verbose=1)

In [ ]:
history = model.fit(X_train, y_train, validation_split=0.1, epochs=3, verbose=1, batch_size=32, callbacks=[tensorboard, checkpoint, reduce_lr])

Epoch 1/3
26/26 [==============================] - ETA: 0s - loss: 0.0395 - accuracy: 0.9914
Epoch 1: val_accuracy did not improve from 0.87778
26/26 [==============================] - 89s 3s/step - loss: 0.0395 - accuracy: 0.9914 - val_loss: 0.4863 - val_accuracy: 0.8667 - lr: 3.0000e-04
Epoch 2/3
26/26 [==============================] - ETA: 0s - loss: 0.0227 - accuracy: 0.9914
Epoch 2: val_accuracy improved from 0.87778 to 0.90000, saving model to effnetB1_V2.h5
26/26 [==============================] - 83s 3s/step - loss: 0.0227 - accuracy: 0.9914 - val_loss: 0.3676 - val_accuracy: 0.9000 - lr: 3.0000e-04
Epoch 3/3
26/26 [==============================] - ETA: 0s - loss: 0.0129 - accuracy: 0.9975
Epoch 3: val_accuracy did not improve from 0.90000
26/26 [==============================] - 82s 3s/step - loss: 0.0129 - accuracy: 0.9975 - val_loss: 0.3293 - val_accuracy: 0.8667 - lr: 3.0000e-04


In [ ]:
def img_pred(path):
    img = cv2.imread(path)
    opencvImage = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    img = cv2.resize(opencvImage,(100,100))
    img = img.reshape(1,100,100,3)
    p = model.predict(img)
    p = np.argmax(p,axis=1)[0]

    if p==0:
        p='Cat'
    else:
        p='Dog'

    if p!=1:
        print(f'The Model predicts that it is a {p}')

In [ ]:
path = r'/content/testes/pug.jpg'
print(img_pred(path))

print("\n")

path = r'/content/testes/cat.jpg'
print(img_pred(path))

1/1 [==============================] - 0s 75ms/step
The Model predicts that it is a Dog
None


1/1 [==============================] - 0s 158ms/step
The Model predicts that it is a Cat
None
